<a href="https://colab.research.google.com/github/paritosh-Shukla24/Autogen_Agents/blob/main/Autogen_Reflection_and_Blogpost_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.1/350.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.8 MB/s eta 0:00:00


In [2]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [11]:
task = '''
        Write a concise but engaging blogpost about
       Nvidia. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [12]:
import os
from google.colab import userdata
openai_api_key=userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_MODEL_NAME']='gpt-3.5-turbo'
os.environ['OPENAI_API_KEY']=openai_api_key

In [13]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [14]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [15]:
print(reply)

Title: "Exploring the Power of Nvidia in the Tech World"

Nvidia isn't just a name; it's a game-changer in the tech industry. Known for its innovative graphics processing units (GPUs) and artificial intelligence applications, Nvidia continues to push boundaries with cutting-edge technologies. From revolutionizing gaming experiences to powering autonomous vehicles, Nvidia's impact is undeniable. Its commitment to driving advancements in data science and machine learning cements its position as a leader in the field. With a focus on performance, efficiency, and scalability, Nvidia is shaping the future of technology in ways we never thought possible. Stay tuned for what they have in store next!


## Adding reflection

Create a critic agent to reflect on the work of the writer agent.

In [16]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Nvidia. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing Innovation: Nvidia's Impact on the Tech Industry"

Nvidia, a powerhouse in the tech world, has been revolutionizing the gaming and artificial intelligence industries with its cutting-edge products and innovative technologies. From high-performance graphics cards to breakthrough advancements in deep learning, Nvidia continues to push boundaries and shape the future of technology. Its relentless pursuit of excellence has earned the company a leading position in the market, with a loyal following of tech enthusiasts and professionals alike. As Nvidia continues to pave the way for groundbreaking advancements, it's clear that its impact on the tech industry is nothing short of extraordinary.

---------------------------------

## Nested chat

In [18]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [19]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [20]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [21]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [22]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


In [23]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [24]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Nvidia. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing Innovation: Nvidia's Impact on the Tech Industry"

Nvidia, a powerhouse in the tech world, has been revolutionizing the gaming and artificial intelligence industries with its cutting-edge products and innovative technologies. From high-performance graphics cards to breakthrough advancements in deep learning, Nvidia continues to push boundaries and shape the future of technology. Its relentless pursuit of excellence has earned the company a leading position in the market, with a loyal following of tech enthusiasts and professionals alike. As Nvidia continues to pave the way for groundbreaking advancements, it's clear that its impact on the tech industry is nothing short of extraordinary.

---------------------------------

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:1264: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

As a legal reviewer, my role is to ensure that the content complies with legal standards and regulations. 

- Avoid making unsubstantiated claims about Nvidia's products or technologies to prevent potential false advertising issues.
- Consider including a disclaimer stating that the impact described is based on opinion and not a factual assertion, to avoid potential misleading advertising claims.
- If mentioning specific advancements or achievements, ensure they are accurately portrayed to prevent any claims of deceptive marketing practices.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Ethics Reviewer):

Review the following content. 
            

 Title: "Unleashing Innovation: Nvidia's Impact on the Tech Industry"

Nvidia,

## Get the summary

In [25]:
print(res.summary)

Title: "Nvidia: A Tech Trailblazer Shaping Tomorrow's World"

In the realm of innovation, Nvidia stands as a towering figure, reshaping the tech landscape with its groundbreaking products and technologies. From unrivaled graphics cards to advancements in AI, Nvidia continues to push boundaries and set new industry standards. With a steadfast commitment to excellence, Nvidia has garnered a dedicated following within the tech community. As Nvidia forges ahead in pioneering advancements, its influence on the industry is profound and unparalleled. In the ever-evolving tech sphere, Nvidia remains a shining beacon of innovation, poised to shape the future and redefine what is possible.
